# Voltage glitching on Arduino UNO

In [3]:
SCOPETYPE = 'CWNANO'
PLATFORM = 'CWNANO'
SS_VER = 'SS_VER_2_1'

In [ ]:
# %run "../../Setup_Scripts/Setup_Generic.ipynb"

## Connect to ChipWhisperer

In [54]:
import chipwhisperer as cw

try:
    if not scope.connectStatus:
        scope.con()
except NameError:
    scope = cw.scope()

try:
    if SS_VER == "SS_VER_2_1":
        target_type = cw.targets.SimpleSerial2
    elif SS_VER == "SS_VER_2_0":
        raise OSError("SS_VER_2_0 is deprecated. Use SS_VER_2_1")
    else:
        target_type = cw.targets.SimpleSerial
except:
    SS_VER="SS_VER_1_1"
    target_type = cw.targets.SimpleSerial

try:
    target = cw.target(scope, target_type)
except:
    print("INFO: Caught exception on reconnecting to target - attempting to reconnect to scope first.")
    print("INFO: This is a work-around when USB has died without Python knowing. Ignore errors above this line.")
    scope = cw.scope()
    target = cw.target(scope, target_type)


print("INFO: Found ChipWhisperer😍")

INFO: Caught exception on reconnecting to target - attempting to reconnect to scope first.
INFO: This is a work-around when USB has died without Python knowing. Ignore errors above this line.
INFO: Found ChipWhisperer😍


In [5]:
if "STM" in PLATFORM or PLATFORM == "CWLITEARM" or PLATFORM == "CWNANO":
    prog = cw.programmers.STM32FProgrammer
elif PLATFORM == "CW303" or PLATFORM == "CWLITEXMEGA":
    prog = cw.programmers.XMEGAProgrammer
elif "neorv32" in PLATFORM.lower():
    prog = cw.programmers.NEORV32Programmer
elif PLATFORM == "CW308_SAM4S" or PLATFORM == "CWHUSKY":
    prog = cw.programmers.SAM4SProgrammer
else:
    prog = None

In [6]:
import time
time.sleep(0.05)
scope.default_setup()

def reset_target(scope):
    if PLATFORM == "CW303" or PLATFORM == "CWLITEXMEGA":
        scope.io.pdic = 'low'
        time.sleep(0.1)
        scope.io.pdic = 'high_z' #XMEGA doesn't like pdic driven high
        time.sleep(0.1) #xmega needs more startup time
    elif "neorv32" in PLATFORM.lower():
        raise IOError("Default iCE40 neorv32 build does not have external reset - reprogram device to reset")
    elif PLATFORM == "CW308_SAM4S" or PLATFORM == "CWHUSKY":
        scope.io.nrst = 'low'
        time.sleep(0.25)
        scope.io.nrst = 'high_z'
        time.sleep(0.25)
    else:  
        scope.io.nrst = 'low'
        time.sleep(0.05)
        scope.io.nrst = 'high_z'
        time.sleep(0.05)

## Firmware for external target usage

To avoid having the STM32F0 GPIOs driven by the CW-Nano when interacting with an external target, set them as high impedance.

In [8]:
%%bash -s "$PLATFORM" "$SS_VER"
cd ./firmware
make PLATFORM=$1 CRYPTO_TARGET=NONE SS_VER=$2 -j

SS_VER set to SS_VER_2_1
SS_VER set to SS_VER_2_1
arm-none-eabi-gcc (15:13.2.rel1-2) 13.2.1 20231009
Copyright (C) 2023 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

mkdir -p objdir-CWNANO 
.
Welcome to another exciting ChipWhisperer target build!!
.
.
.
.
.
Compiling:
.
Compiling:
Compiling:
Compiling:
Compiling:
Assembling: ../../../firmware/mcu/hal//stm32f0/stm32f0_startup.S
arm-none-eabi-gcc -c -mcpu=cortex-m0 -I. -x assembler-with-cpp -mthumb -mfloat-abi=soft -ffunction-sections -DF_CPU=7372800 -Wa,-gstabs,-adhlns=objdir-CWNANO/stm32f0_startup.lst -I../../../firmware/mcu/simpleserial/ -I../../../firmware/mcu/hal/ -I../../../firmware/mcu/hal/ -I../../../firmware/mcu/hal//stm32f0 -I../../../firmware/mcu/hal//stm32f0/CMSIS -I../../../firmware/mcu/hal//stm32f0/CMSIS/core -I../../../firmware/mcu/hal//stm32f0/CMSIS/device -I../../../firmware/mcu/hal//s

/usr/lib/gcc/arm-none-eabi/13.2.1/../../../arm-none-eabi/bin/ld: /usr/lib/gcc/arm-none-eabi/13.2.1/../../../arm-none-eabi/lib/thumb/v6-m/nofp/libg_nano.a(libc_a-closer.o): in function `_close_r':
/build/newlib-38V0JC/newlib-4.4.0.20231231/build_nano/arm-none-eabi/thumb/v6-m/nofp/newlib/../../../../../../newlib/libc/reent/closer.c:47:(.text+0xc): warning: _close is not implemented and will always fail


/usr/lib/gcc/arm-none-eabi/13.2.1/../../../arm-none-eabi/bin/ld: /usr/lib/gcc/arm-none-eabi/13.2.1/../../../arm-none-eabi/lib/thumb/v6-m/nofp/libg_nano.a(libc_a-closer.o): note: the message above does not take linker garbage collection into account
/usr/lib/gcc/arm-none-eabi/13.2.1/../../../arm-none-eabi/bin/ld: /usr/lib/gcc/arm-none-eabi/13.2.1/../../../arm-none-eabi/lib/thumb/v6-m/nofp/libg_nano.a(libc_a-lseekr.o): note: the message above does not take linker garbage collection into account
/usr/lib/gcc/arm-none-eabi/13.2.1/../../../arm-none-eabi/bin/ld: /usr/lib/gcc/arm-none-eabi/13.2.1/../../../arm-none-eabi/lib/thumb/v6-m/nofp/libg_nano.a(libc_a-readr.o): note: the message above does not take linker garbage collection into account


/usr/lib/gcc/arm-none-eabi/13.2.1/../../../arm-none-eabi/bin/ld: /usr/lib/gcc/arm-none-eabi/13.2.1/../../../arm-none-eabi/lib/thumb/v6-m/nofp/libg_nano.a(libc_a-lseekr.o): in function `_lseek_r':
/build/newlib-38V0JC/newlib-4.4.0.20231231/build_nano/arm-none-eabi/thumb/v6-m/nofp/newlib/../../../../../../newlib/libc/reent/lseekr.c:49:(.text+0x10): warning: _lseek is not implemented and will always fail
/usr/lib/gcc/arm-none-eabi/13.2.1/../../../arm-none-eabi/bin/ld: /usr/lib/gcc/arm-none-eabi/13.2.1/../../../arm-none-eabi/lib/thumb/v6-m/nofp/libg_nano.a(libc_a-readr.o): in function `_read_r':
/build/newlib-38V0JC/newlib-4.4.0.20231231/build_nano/arm-none-eabi/thumb/v6-m/nofp/newlib/../../../../../../newlib/libc/reent/readr.c:49:(.text+0x10): warning: _read is not implemented and will always fail
/usr/lib/gcc/arm-none-eabi/13.2.1/../../../arm-none-eabi/bin/ld: /usr/lib/gcc/arm-none-eabi/13.2.1/../../../arm-none-eabi/lib/thumb/v6-m/nofp/libg_nano.a(libc_a-writer.o): in function `_write_r'

/usr/lib/gcc/arm-none-eabi/13.2.1/../../../arm-none-eabi/bin/ld: /usr/lib/gcc/arm-none-eabi/13.2.1/../../../arm-none-eabi/lib/thumb/v6-m/nofp/libg_nano.a(libc_a-writer.o): note: the message above does not take linker garbage collection into account
-e Done!
.
.
.
.
.
Creating load file for Flash: arduino_voltage_glitching-CWNANO.hex
Creating load file for Flash: arduino_voltage_glitching-CWNANO.bin
arm-none-eabi-objcopy -O ihex -R .eeprom -R .fuse -R .lock -R .signature arduino_voltage_glitching-CWNANO.elf arduino_voltage_glitching-CWNANO.hex
arm-none-eabi-objcopy -O binary -R .eeprom -R .fuse -R .lock -R .signature arduino_voltage_glitching-CWNANO.elf arduino_voltage_glitching-CWNANO.bin
Creating load file for EEPROM: arduino_voltage_glitching-CWNANO.eep
Creating Extended Listing: arduino_voltage_glitching-CWNANO.lss
arm-none-eabi-objcopy -j .eeprom --set-section-flags=.eeprom="alloc,load" \
--change-section-lma .eeprom=0 --no-change-warnings -O ihex arduino_voltage_glitching-CWNANO.e

In [11]:
fw_path = f"./firmware/arduino_voltage_glitching-{PLATFORM}.hex"
cw.program_target(scope, prog, fw_path)

Detected known STMF32: STM32F04xxx
Extended erase (0x44), this can take ten seconds or more
Attempting to program 2455 bytes at 0x8000000
STM32F Programming flash...
STM32F Reading flash...
Verified flash OK, 2455 bytes


## Attack

In [58]:
import chipwhisperer as cw
import serial
import serial.tools.list_ports
import time

cw.set_all_log_levels(cw.logging.CRITICAL)

try:
    # print(scope)

    # Setup serial communication
    ser_port = ""
    print(f"{scope.get_serial_ports()[0]["port"]}: ChipWhisperer USB port")
    ports = serial.tools.list_ports.comports()
    for port, desc, _ in sorted(ports):
        if "CH340" in desc:
            print(f"{port}: {desc}\n")
            ser_port = port
    ser = serial.Serial(ser_port, 9600, timeout=.1)  # adjust port as needed

    reset_target(scope)

    # Send a wrong password
    ser.write(b"password\n")
    time.sleep(0.05)
    ser.write(b"\n")
    # Read all response lines
    for line in ser.readlines():
        print(line.decode().strip())

finally:
    ser.close()

cw.set_all_log_levels(cw.logging.WARNING)

COM3: ChipWhisperer USB port
COM7: USB-SERIAL CH340 (COM7)

>> ACCESS DENIED

Enter password:
No input. Enter password:


In [52]:
scope.dis()
target.dis()